# Goals
* Predict Results of Pro matches with 70% accuracy

### Data that will be used
* match data and maps

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import glob

#combines all the csvs into one table
list_of_csv_files = glob.glob("datasets"+ "/*.csv")
list_of_csv_files.sort()
df = pd.concat(map(pd.read_csv, list_of_csv_files), ignore_index=True)
df

,Unnamed: 0,round,winningTeamRound,losingTeamRound,date,map,event,winner,loser,matchid
0,0,1,0,3,17-09-21,Ascent,Valorant Champions Tour Stage 3: Masters Berlin,100 Thieves,Acend,100 ThievesvsAcend17-09-21
1,1,2,0,3,17-09-21,Ascent,Valorant Champions Tour Stage 3: Masters Berlin,100 Thieves,Acend,100 ThievesvsAcend17-09-21
2,2,3,1,0,17-09-21,Ascent,Valorant Champions Tour Stage 3: Masters Berlin,100 Thieves,Acend,100 ThievesvsAcend17-09-21
3,3,4,1,0,17-09-21,Ascent,Valorant Champions Tour Stage 3: Masters Berlin,100 Thieves,Acend,100 ThievesvsAcend17-09-21
4,4,5,1,0,17-09-21,Ascent,Valorant Champions Tour Stage 3: Masters Berlin,100 Thieves,Acend,100 ThievesvsAcend17-09-21
...,...,...,...,...,...,...,...,...,...,...
75412,18,19,2,0,12-03-21,Split,Champions Tour Turkey Stage 1: Masters,İstanbul Wildcats,Zero Zone,İstanbul WildcatsvsZero Zone12-03-21
75413,19,20,1,0,12-03-21,Split,Champions Tour Turkey Stage 1: Masters,İstanbul Wildcats,Zero Zone,İstanbul WildcatsvsZero Zone12-03-21
75414,20,21,0,3,12-03-21,Split,Champions Tour Turkey Stage 1: Masters,İstanbul Wildcats,Zero Zone,İstanbul WildcatsvsZero Zone12-03-21
75415,21,22,0,3,12-03-21,Split,Champions Tour Turkey Stage 1: Masters,İstanbul Wildcats,Zero Zone,İstanbul WildcatsvsZero Zone12-03-21


In [16]:
csv = pd.read_csv('playerData/vlr.csv')
matches = csv['matchID'].unique()
csv[(csv['matchID']==matches[3])&(csv['map']=='Icebox')]

,playerName,playerTeam,map,playerAgent,playerKills,playerDeaths,playerAssists,playerACS,playerADR,playerHS,playerFirstBlood,playerFirstDeath,matchID,opponent,result,winningTeam,date
32,mwzera,GLB,Icebox,raze,25,19,4,25,183.0,0.0,4.0,0.0,GLBvsSLK25-07-21,GLB,Win,GLB,25-07-21
47,BELKY,GLB,Icebox,astra,22,14,3,22,169.0,2.0,3.0,2.0,GLBvsSLK25-07-21,GLB,Win,GLB,25-07-21
48,Jonn,GLB,Icebox,reyna,18,19,6,18,151.0,4.0,4.0,4.0,GLBvsSLK25-07-21,GLB,Win,GLB,25-07-21
49,fznnn,GLB,Icebox,sova,15,14,6,15,133.0,0.0,1.0,0.0,GLBvsSLK25-07-21,GLB,Win,GLB,25-07-21
50,Nyang,GLB,Icebox,sage,12,16,7,12,109.0,4.0,1.0,4.0,GLBvsSLK25-07-21,GLB,Win,GLB,25-07-21
72,aspas,SLK,Icebox,jett,23,16,3,23,169.0,3.0,2.0,3.0,GLBvsSLK25-07-21,SLK,Lose,GLB,25-07-21
73,DiMAS,SLK,Icebox,viper,19,18,8,19,139.0,2.0,0.0,2.0,GLBvsSLK25-07-21,SLK,Lose,GLB,25-07-21
74,Maverick,SLK,Icebox,kayo,13,20,15,13,125.0,3.0,2.0,3.0,GLBvsSLK25-07-21,SLK,Lose,GLB,25-07-21
75,mNdS,SLK,Icebox,sage,14,18,4,14,103.0,4.0,3.0,4.0,GLBvsSLK25-07-21,SLK,Lose,GLB,25-07-21
101,Veroneze,SLK,Icebox,sova,13,20,3,13,129.0,1.0,3.0,1.0,GLBvsSLK25-07-21,SLK,Lose,GLB,25-07-21
